In [16]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

# Load and process the text data
filepath = tf.keras.utils.get_file('shakes.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

# Read the text file, decode to UTF-8, and convert to lowercase
text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()

# Define parameters
seq_length = 40
STEP_SIZE = 3

# Create character mapping
characters = sorted(set(text))
char_to_index = {c: i for i, c in enumerate(characters)}
index_to_char = {i: c for i, c in enumerate(characters)}

# Prepare input and output sequences
sentences = []
next_characters = []

for i in range(0, len(text) - seq_length, STEP_SIZE):
    sentences.append(text[i:i + seq_length])
    next_characters.append(text[i + seq_length])

# One-hot encode the input and output data
x = np.zeros((len(sentences), seq_length, len(characters)), dtype=bool)
y = np.zeros((len(sentences), len(characters)), dtype=bool)

for i, sentence in enumerate(sentences):
    for t, character in enumerate(sentence):
        x[i, t, char_to_index[character]] = 1  # One-hot encode characters in the sequence
    y[i, char_to_index[next_characters[i]]] = 1  # One-hot encode the next character

# Build the LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(seq_length, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01))

# Print shapes of x and y
print(x.shape)
print(y.shape)

# Train the model
model.fit(x, y, batch_size=256, epochs=4)

# Save the model


(371785, 40, 39)
(371785, 39)
Epoch 1/4
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 116s 80ms/step - loss: 2.2496
Epoch 2/4
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 118s 81ms/step - loss: 1.6118
Epoch 3/4
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 115s 79ms/step - loss: 1.5112
Epoch 4/4
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 115s 79ms/step - loss: 1.4586


In [2]:
model.save('poetic_text_generator.keras')


In [12]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-8) / temperature  # Apply temperature scaling
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)  # Normalize to get probabilities
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


In [13]:
def generate_text(length , temperature):
    start : random.randin(0 , len(text)- seq-length-1)

In [21]:
def generate_text(length, temperature=1.0):
    # Ensure you start with a valid seed
    start_index = np.random.randint(0, len(text) - seq_length - 1)
    sentence = text[start_index: start_index + seq_length]
    generated = sentence

    # Generate `length` characters
    for i in range(length):
        # Create the input array (1, seq_length, len(characters))
        x = np.zeros((1, seq_length, len(characters)))
        
        # One-hot encode the current sentence
        for t, char in enumerate(sentence):
            x[0, t, char_to_index[char]] = 1
        
        # Predict the next character probabilities
        predictions = model.predict(x, verbose=0)[0]
        
        # Sample the next character based on temperature
        next_index = sample(predictions, temperature)
        next_char = index_to_char[next_index]
        
        # Add the next character to the generated text
        generated += next_char
        
        # Update the input sequence (shift left by 1)
        sentence = sentence[1:] + next_char
    
    return generated  # Return the generated text


In [22]:
print('---------0.2---------')
print(generate_text(300, 0.2))

print('---------0.4---------')
print(generate_text(300, 0.4))

print('---------0.6---------')
print(generate_text(300, 0.6))

print('---------0.8---------')
print(generate_text(300, 0.8))

print('---------1---------')
print(generate_text(300, 1))


---------0.2---------
me;
and, by my troth, you have cause. your father.

coriolanus:
what, sir, i have mercutio, and so the death,
and so shall be the son the stander of the poor soul to the man
and that be this shall say i have so in the stand.

buttaret:
he shall say the son to the hands of his hand.

coriolanus:
the brother to my son the stand to man all t
---------0.4---------
 language! heavens!
i am the best of the greater latence his
a what i may sleep the man all the sword
than our england the own child the crown of will
to so that say i think the father to your noble conscanor,
and what do i have the crown to be this place.

marcius:
what swell me his honour will bring with his myself
and say the roment of
---------0.6---------
not pay for the glasses you have burst?

complayih:
art is on the stone by good for a grandess will part
the fall of my courtry up.

first senator:
sir, yet i did, a byour chair words,
that well plase me sleep'd and death them:
how now i see, and well 

In [25]:
start_text = "In the quiet of the night, "  # Your seed text
generated_text = generate_text( length=300, temperature=0.5)
print("Generated Text:\n", generated_text)

Generated Text:
  heat of blood.
and lack of temper'd judget your honour confect
to his peace and reaton'd the supptition's after.

mercutio:
what, marricy have i do the man and the store.

second servant:
these word here of rowars, at his swearful:
when i have not thy son horses worship,
what the son to the courted all of him.

aufidius:
what, sir here, 
